## Dynamic Plans

This notebook implements a uav logic instead of a plan and decisions are done online following that logic

In [ ]:
import os
import subprocess
import numpy as np


# Custom modules
from vehicle_logic import VehicleLogic
from helpers.change_coordinates import heading_to_yaw,find_spawns,global2local
from helpers.visualization.gazebo_world import update_world
from helpers.navegation_logic import find_path
from helpers.visualization.plots import plot_3d_interactive
from gcs import GCS
from plan.core import State
from plan import Plan

Kill all related process

In [2]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

Paths

In [3]:
QGC_path='~/QGroundControl.AppImage'
gazebo_word_path="gazebo_worlds/medical_delivery.world" ##
ardupilot_vehicle_path='~/ardupilot/Tools/autotest/sim_vehicle.py'

# Choose Simulator

In [11]:
simulator ='gazebo' # 'QGroundControl' #

## Gonfiguration

In [12]:
target_box_center=(-85.263400,56.518300, 0.593695, 0)
origin=(0,0,0)
offsets = [ #east, north, up, heading
    target_box_center, 
    (0,-10, 0, 0),
    (1,-15, 0, 0),
    (2,-10, 0, 0)]

n_uavs = len(offsets)

wps = np.array([(40, -10, 5),
                (15, -10, 5),
                (0, -10, 5),
                (0,   4, 5),
                (40, 20, 5),
                (15, 20, 5),
                (0,  20, 5)])

homes=np.array([offset[:3] for offset in offsets])
target_wps=np.array([
                (0, 4,5),
                (0, 4,5),
                (0, 4,5),
                (0, 4,5)])

gcs=GCS(positions=homes.copy())

targets=np.array([(*target_wp[:2],0) for target_wp in target_wps])
markers = {}

# Interactive widget to rotate the plot
if markers:
    plot_3d_interactive(markers,title='Simulation Markers',expand=[0.2,0.2,0.4],ground=-0.05)


if simulator == 'QGroundControl':
    # lat,long,alt,heading
    home_position=(-35.3633245,149.1652241,0,90)
    spawns=find_spawns(home_position, offsets)

## Get Local positions

In [13]:
local_wps=global2local(wps, homes,pairwise=False)

Launch Simulator

In [14]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser(QGC_path)
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    drones = [(east, north, up, 0, 0, heading_to_yaw(heading)) for i, (east, north, up, heading) in enumerate(offsets)]
    world_path = os.path.expanduser(gazebo_word_path)
    updated_world_path = update_world(drones,markers,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 

simulator_process = subprocess.Popen(
                    sim_cmd,
                    stdout=subprocess.DEVNULL,  # Suppress standard output
                    stderr=subprocess.DEVNULL,  # Suppress error output
                    shell=False  # Ensure safety when passing arguments
                    )


Launch Vehicle

In [15]:
vehicle_path = os.path.expanduser(ardupilot_vehicle_path)
for i in range(n_uavs):
    vehicle_cmd = f"python3 {vehicle_path} -v ArduCopter -I{i} --sysid {i+1} --no-rebuild"
    if simulator == 'QGroundControl':
        spawn=','.join(map(str, spawns[i]))
        vehicle_cmd += f" --custom-location={spawn}"
    elif simulator == 'gazebo':
        vehicle_cmd += " -f gazebo-iris"
    subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

Create uavs

In [16]:
alts=5*[5,5,5,7]#10.5
uavs=[]
for i,home in enumerate(zip(homes)):
    uavs.append(VehicleLogic(sys_id=i+1,
                            home=home,
                            plan=Plan.hover(alt=alts[i])))

Plan 'hover' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'takeoff' created — no connection yet 🧩
Step 'takeoff' created — no connection yet 🧩
Action 'fly' created — no connection yet 🧩
Vehicle 1: Step 'Check disarmed' is now connected ✅🔗
Vehicle 1: Step 'Check EKF' is now connected ✅🔗
Vehicle 1: Step 'Check GPS' is now connected ✅🔗
Vehicle 1: Step 'Check system' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Step 'arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is no

## Execute Plan

In [17]:
uavs[3].plan

🚀 <Plan 'hover'>
  🕓 <Action 'Pre-Arm Check'>
    🕓 <Step 'Check disarmed'>
    🕓 <Step 'Check EKF'>
    🕓 <Step 'Check GPS'>
    🕓 <Step 'Check system'>
  🕓 <Action 'Arm'>
    🕓 <Step 'arm'>
  🕓 <Action 'Set Mode: GUIDED'>
    🕓 <Step 'Switch to GUIDED'>
  🕓 <Action 'takeoff'>
    🕓 <Step 'takeoff'>
  🕓 <Action 'fly'>

In [ ]:
while gcs.incomplete_missions:
    to_remove = set()
    for id in gcs.incomplete_missions:
        i = id-1
        uav=uavs[i]
        uav.plan.act()
        if uav.plan.state in [State.DONE,State.FAILED]:
            to_remove.add(id)
    gcs.update_missions(to_remove)

Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 2: ▶️ Starting Step: Check disarmed
Vehicle 3: ▶️ Starting Step: Check disarmed
Vehicle 4: ▶️ Starting Step: Check disarmed
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 2: ✅ Step: Check disarmed is done
Vehicle 3: ✅ Step: Check disarmed is done
Vehicle 4: ✅ Step: Check disarmed is done
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 2: ▶️ Starting Step: Check EKF
Vehicle 3: ▶️ Starting Step: Check EKF
Vehicle 4: ▶️ Starting Step: Check EKF
Vehicle 4: ✅ Step: Check EKF is done
Vehicle 4: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: ▶️ Starting Step: Check GPS
Vehicle 4: ✅ Step: Check GPS is done
Vehicle 4: ▶️ Starting Step: Check system
Vehicle 2: ✅ Step: Check EKF is done
Vehicle 2: ▶️ Starting Step: Check GPS
Vehicle 3: ✅ Step: Check EKF is done
Vehicle 3: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check GPS is done
Vehicle 1: ▶️ Starting Step: Check system
Vehicle 4: ✅ Step: Check system is don

## Kill all related process

In [ ]:
# for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
#     os.system(f"pkill -9 -f {process}")